In [1]:
# coding: utf-8
from typing import Dict, Optional

import pandas as pd
import plotly.express as px

from data.data_loader import PykrxDataLoader
from factor.factor_strategy import fundamental_adjuster
from simulation.account import Account
from simulation.broker import Broker
from simulation.metric import cagr, mdd, sharpe_ratio, sortino_ratio
from simulation.utility import get_lookback_fromdate, rebalance

## 1. 팩터 투자

### 1.1. 유틸리티 함수 정의

In [2]:
def date_adjust(index_df: pd.DataFrame, df: pd.DataFrame):
    new_index = downsample_df(index_df).index
    # 원래 인덱스에 새 인덱스 삽입
    df.index = df.index.map(lambda x: x.replace(day=new_index[(new_index.year == x.year) & (new_index.month == x.month)][0].day))
    return df

def downsample_df(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()
    data.index = pd.to_datetime(data.index)
    data['date'] = data.index
    data = data.resample('M').apply('last')

    return data.set_index(pd.DatetimeIndex(data.date)).drop(columns=['date'])

### 1.2. 팩터 전략 정의하기

#### 1.2.1. 모멘텀(상대) 전략 정의하기

In [3]:
def calculate_momentum(ohlcv_data: pd.DataFrame,
                       lookback_period: int,
                       skip_period: int) -> pd.DataFrame:
    # 데이터 재구조화
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(['ticker', 'date']).unstack(level=0)['close']

    # 팩터 계산(모멘텀)
    momentum_data = data.shift(periods=skip_period).rolling(window=lookback_period).apply(lambda x: x[-1] / x[0] - 1)

    return momentum_data

#### 1.2.2. 가치(PER, PBR) 및 배당 전략 정의하기

In [4]:
def calculate_fundamental(ohlcv_data: pd.DataFrame,
                          market_cap_data: pd.DataFrame,
                          fundamental_data: pd.DataFrame,
                          strategy_name: str,
                          lookback_period: int = 12, ) -> pd.DataFrame:

    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 조정 및 재구조화(기본)
    fundamental_data = fundamental_adjuster(fundamental_data=fundamental_data, market_cap_data=market_cap_data)
    mapping = {'per': 'EPS', 'pbr': 'BPS', 'dividend': 'DPS'}
    target_fundamental = mapping.get(strategy_name)
    fundamental_data = fundamental_data[
        ['date', target_fundamental, 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)[target_fundamental]

    # 팩터 계산(PER, PBR, 배당)
    if strategy_name == 'per':
        fundamental_data = fundamental_data.rolling(window=lookback_period).sum()
        fundamental_data = data / fundamental_data
    elif strategy_name == 'pbr':
        fundamental_data = data / fundamental_data
    elif strategy_name == 'dividend':
        fundamental_data = fundamental_data / data
    else:
        raise ValueError

    return fundamental_data

def fundamental_adjuster(fundamental_data: pd.DataFrame,
                      market_cap_data: pd.DataFrame) -> pd.DataFrame:
    # 현재 주식 수로 나누기
    market_cap_data['shares_div'] = market_cap_data['shares'].div(
        market_cap_data.groupby('ticker')['shares'].transform('last')
    )
    market_cap_data = market_cap_data[['shares_div', 'ticker']].reset_index()

    # 조정
    data = fundamental_data.reset_index().merge(
        market_cap_data, on=['date', 'ticker']).set_index(['date'])
    data['EPS'] = data['shares_div'] * data['EPS']
    data['BPS'] = data['shares_div'] * data['BPS']
    data['DPS'] = data['shares_div'] * data['DPS']

    # 주식 수 바뀐 당일 오류 탐색
    changed_row = data.copy()
    changed_row['previous'] = changed_row.shares_div.shift(1)
    changed_row = changed_row[changed_row.shares_div != changed_row.previous]
    changed_row = changed_row.reset_index().set_index(['ticker', 'date'])

    # 당일 오류 결정
    data = data.reset_index().set_index(['ticker', 'date'])
    for i, index in enumerate(data.index):
        if index in changed_row.index:
            data.loc[index] = data.iloc[i+1]

    return data.reset_index()[['ticker', 'date', 'BPS', 'EPS', 'DPS']]

#### 1.2.3. 소형주 전략 정의하기

In [5]:
def calculate_small(ohlcv_data: pd.DataFrame,
                    market_cap_data: pd.DataFrame) -> pd.DataFrame:
    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 재구조화(주식 수)
    market_cap_data = market_cap_data[['shares',
                                       'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['shares']

    # 시장 데이터 날짜 수정
    market_cap_data = date_adjust(index_df=data, df=market_cap_data)

    # 팩터 계산(시가총액)
    market_cap_data = market_cap_data * data

    return market_cap_data

### 1.3. 시뮬레이션을 위한 함수 정의하기

In [6]:
def get_factor_weight(factor_data: pd.DataFrame,
                      buying_ratio: float,
                      strategy_name: str) -> Optional[Dict]:
    # 데이터 중 결측치가 있는지 확인함
    if factor_data.isnull().values.any():
        return None

    # 매수 주식 선정
    reverse = {'per', 'pbr', 'small', 'lowvol'}
    ratio = buying_ratio if strategy_name in reverse else 1 - buying_ratio
    top_quantile = factor_data.quantile(ratio)
    if strategy_name in reverse:
        stocks_to_buy = factor_data[factor_data <= top_quantile].index.to_list()
    else:
        stocks_to_buy = factor_data[factor_data >= top_quantile].index.to_list()

    # 주식 비율 할당
    weights = 1 / len(stocks_to_buy) if stocks_to_buy else 0
    portfolio = {ticker: weights if ticker in stocks_to_buy else 0.0 for ticker in factor_data.index}

    return portfolio

### 1.4. 팩터 투자 전략을 이용한 시뮬레이션 함수 정의하기

In [7]:
def simulate_factor(ohlcv_data: pd.DataFrame,
                    market_cap_data: Optional[pd.DataFrame],
                    fundamental_data: Optional[pd.DataFrame],
                    trader_data: Optional[pd.DataFrame],
                    lookback_period: Optional[int],
                    skip_period: Optional[int],
                    strategy_name: str,
                    buying_ratio: float = 0.1) -> Account:
    # 계좌 및 브로커 선언
    account = Account(initial_cash=100000000)
    broker = Broker()

    # 팩터 계산
    if strategy_name == 'relative':
        factor_data = calculate_momentum(ohlcv_data=ohlcv_data, lookback_period=lookback_period, skip_period=skip_period,)
    elif strategy_name in ['per', 'pbr', 'dividend']:
        factor_data = calculate_fundamental(ohlcv_data=ohlcv_data,
                                            market_cap_data=market_cap_data,
                                            fundamental_data=fundamental_data,
                                            lookback_period=lookback_period,
                                            strategy_name=strategy_name)
    elif strategy_name == 'small':
        factor_data = calculate_small(ohlcv_data=ohlcv_data,
                                      market_cap_data=market_cap_data)
    elif strategy_name in {'indivisual', 'institutional', 'foreign'}:
        factor_data = calculate_trader(ohlcv_data=ohlcv_data,
                                       market_cap_data=market_cap_data,
                                       trader_data=trader_data,
                                       lookback_period=lookback_period,
                                       strategy_name=strategy_name)
    elif strategy_name == 'lowvol':
        factor_data = caculate_lowvol(ohlcv_data=ohlcv_data,
                                      lookback_period=lookback_period)
    else:
        raise ValueError

    for date, ohlcv in ohlcv_data.groupby('date'):
        # 주문 집행 및 계좌 갱신
        transactions = broker.process_order(dt=date, data=ohlcv, orders=account.orders)
        account.update_position(transactions=transactions)
        account.update_portfolio(dt=date, data=ohlcv)

        # 팩터 전략을 이용해 포트폴리오 구성
        factor_data_slice = factor_data.loc[date]
        weights = get_factor_weight(factor_data=factor_data_slice,
                                    buying_ratio=buying_ratio,
                                    strategy_name=strategy_name)

        print(f'Portfolio: {weights}')
        if weights is None:
            continue

        # 주문 생성
        rebalance(dt=date, data=ohlcv, account=account, weights=weights)

    return account

### 1.5. 데이터 불러오기

In [8]:
# 데이터 시작과 끝 날짜 정의
fromdate = '2013-04-01'
todate = '2021-12-31'

# 투자할 종목 후보 정의
ticker_list = ['000660', '005490', '051910', '006400', '005380', '000270',
               '012330', '068270', '105560', '096770', '055550', '066570',
               '047050', '032830', '015760', '086790', '000810', '033780',
               '034730', '034020', '009150', '138040', '010130', '001570',
               '010950', '024110', '030200', '051900', '009830', '086280',
               '011170', '011070', '012450', '036570', '005830', '161390',
               '034220', '004020', '032640', '097950', '000720', '006800',
               '006260', '010620', '011780', '078930', '005940', '029780',
               '128940', '035250', '016360', '021240', '010120', '052690',
               '008770', '071050', '000990', '001450', '020150', '039490',
               '111770', '000880', '004370', '036460', '007070', '138930',
               '139480', ]

# 데이터 불러오기
data_loader = PykrxDataLoader(fromdate=fromdate, todate=todate, market="KOSPI")
# ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list, freq='m',
#                                          delay=1)
# ohlcv_data_day = data_loader.load_stock_data(ticker_list=ticker_list, freq='d',
#                                              delay=1)
# fundamental_data = data_loader.load_fundamental_data(ticker_list=ticker_list,
#                                                      freq='m', delay=1)
# market_cap_data = data_loader.load_market_cap_data(ticker_list=ticker_list,
#                                                    freq='m', delay=1)
# trader_data = data_loader.load_trader_data(ticker_list=ticker_list, freq='m',
#                                            delay=1)
ohlcv_data = pd.read_csv('./factor/data/ohlcv_data.csv', dtype={'ticker': str})
ohlcv_data['date'] = pd.to_datetime(ohlcv_data['date'])
ohlcv_data.set_index('date', inplace=True)

ohlcv_data_day = pd.read_csv('./factor/data/ohlcv_data_day.csv', dtype={'ticker': str})
ohlcv_data_day['date'] = pd.to_datetime(ohlcv_data_day['date'])
ohlcv_data_day.set_index('date', inplace=True)

fundamental_data = pd.read_csv('./factor/data/fundamental_data.csv', dtype={'ticker': str})
fundamental_data['date'] = pd.to_datetime(fundamental_data['date'])
fundamental_data.set_index('date', inplace=True)

market_cap_data = pd.read_csv('./factor/data/market_cap_data.csv', dtype={'ticker': str})
market_cap_data['date'] = pd.to_datetime(market_cap_data['date'])
market_cap_data.set_index('date', inplace=True)

trader_data = pd.read_csv('./factor/data/trader_data.csv', dtype={'ticker': str})
trader_data['date'] = pd.to_datetime(trader_data['date'])
trader_data.set_index('date', inplace=True)

# 데이터 확인하기
print(f'주가 데이터: {ohlcv_data.shape}, 기본 데이터: {fundamental_data.shape}, '
      f'주식 수 데이터: {market_cap_data.shape}, 수급 주체 데이터: {trader_data.shape}')

주가 데이터: (6568, 6), 기본 데이터: (7035, 7), 주식 수 데이터: (7035, 5), 수급 주체 데이터: (7035, 6)


### 1.6. 전략 실행하기

In [9]:
# 시뭎레이션 시작일
simulation_fromdate = '2017-02-01'

In [10]:
def df_slicer(df: pd.DataFrame, fromdate: str):
    frequency = df.frequency if hasattr(df, 'frequenccy') else None
    sliced_df = df[fromdate:]
    sliced_df.__setattr__('frequency', frequency)

    return sliced_df

#### 1.6.1. 상대모멘텀 전략 실행하기

In [11]:
# 룩백 기간, 생략 기간 정의
lookback = 3
offset = 1
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'relative'

# 상대모멘텀 전략  실행하기
account_relative = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=None,
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio
)

/var/folders/rk/v4lbk0_j1l5ddg0wmz4352880000gn/T/ipykernel_54041/935161074.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  momentum_data = data.shift(periods=skip_period).rolling(window=lookback_period).apply(lambda x: x[-1] / x[0] - 1)


Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfoli

#### 1.6.2. PER 가치 전략 실행하기

In [12]:
# 룩백 기간, 생략 기간 정의
lookback = 12
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'per'

# PER 가치 전략 실행하기
account_per = simulate_factor(ohlcv_data=ohlcv_data_day[total_lookback:],
                              market_cap_data=market_cap_data[total_lookback:],
                              fundamental_data=fundamental_data[total_lookback:],
                              trader_data=None,
                              lookback_period=lookback,
                              skip_period=offset,
                              strategy_name=strategy,
                              buying_ratio=ratio)

Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfoli

#### 1.6.3. PBR 가치 전략 실행하기

In [13]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'pbr'

# PBR 가치 전략 실행하기
account_pbr = simulate_factor(ohlcv_data=ohlcv_data_day[total_lookback:],
                              market_cap_data=market_cap_data[total_lookback:],
                              fundamental_data=fundamental_data[total_lookback:],
                              trader_data=None,
                              lookback_period=lookback,
                              skip_period=offset,
                              strategy_name=strategy,
                              buying_ratio=ratio)

Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.14285714285714285, '004370': 0.0, '005380': 0.0, '005490': 0.14285714285714285, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.14285714285714285, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.14285714285714285, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.14285714285714285, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.14285714285714285, '096770': 0.0, '097950

### 1.6.4 배당 전략 실행하기

In [14]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'dividend'

# 배당 전략 실행하기
account_dividend = simulate_factor(
    ohlcv_data=ohlcv_data[total_lookback:],
    market_cap_data=market_cap_data[total_lookback:],
    fundamental_data=fundamental_data[total_lookback:],
    trader_data=None,
    lookback_period=lookback,
    skip_period=offset,
    strategy_name=strategy,
    buying_ratio=ratio
)

Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.14285714285714285, '006260': 0.0, '006400': 0.0, '006800': 0.14285714285714285, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.14285714285714285, '016360': 0.0, '020150': 0.0, '021240': 0.14285714285714285, '024110': 0.0, '029780': 0.14285714285714285, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.14285714285714285, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.14285714285714285, '086790': 0.0, '09677

### 1.6.5 소형주 전략 실행

In [15]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'small'

# 소형주 전략 실행하기
account_small = simulate_factor(
    ohlcv_data=ohlcv_data[total_lookback:],
    market_cap_data=market_cap_data[total_lookback:],
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback,
    skip_period=offset,
    strategy_name=strategy,
    buying_ratio=ratio
)

Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.14285714285714285, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.14285714285714285, '010130': 0.0, '010620': 0.14285714285714285, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.14285714285714285, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.14285714285714285, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '096770': 0.0, '097950

/var/folders/rk/v4lbk0_j1l5ddg0wmz4352880000gn/T/ipykernel_54041/3946629328.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.resample('M').apply('last')


Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.14285714285714285, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.14285714285714285, '010130': 0.0, '010620': 0.14285714285714285, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.14285714285714285, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.14285714285714285, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '096770': 0.0, '097950

### 히스토리 형식 변환

In [16]:
per_account = pd.DataFrame(account_per.account_history).set_index('date')
pbr_account = pd.DataFrame(account_pbr.account_history).set_index('date')
dividend_account = pd.DataFrame(account_dividend.account_history).set_index('date')
small_account = pd.DataFrame(account_small.account_history).set_index('date')

### 수익률 계산

In [17]:
per_returns = per_account['total_asset'].resample('ME').last().pct_change().loc[simulation_fromdate:]
per_returns.name = 'per_return'
per_returns.head()

date
2017-02-28    0.000000
2017-03-31    0.028667
2017-04-30   -0.005687
2017-05-31    0.056259
2017-06-30    0.018808
Freq: ME, Name: per_return, dtype: float64

In [18]:
pbr_returns = pbr_account['total_asset'].resample('ME').last().pct_change().loc[simulation_fromdate:]
pbr_returns.name = 'pbr_return'
pbr_returns.iloc[0] = 0.0
pbr_returns.head()

date
2017-02-28    0.000000
2017-03-31   -0.011505
2017-04-30   -0.000124
2017-05-31    0.041399
2017-06-30    0.034310
Freq: ME, Name: pbr_return, dtype: float64

In [19]:
dividend_returns = dividend_account['total_asset'].resample('ME').last().pct_change().loc[simulation_fromdate:]
dividend_returns.name = 'dividend_return'
dividend_returns.iloc[0] = 0.0
dividend_returns.head()

date
2017-02-28    0.000000
2017-03-31    0.001618
2017-04-30    0.014855
2017-05-31    0.053914
2017-06-30   -0.027128
Freq: ME, Name: dividend_return, dtype: float64

In [27]:
small_returns = small_account['total_asset'].resample('ME').last().pct_change().loc[simulation_fromdate:]
small_returns.name = 'small_return'
small_returns.iloc[0] = 0.0
small_returns.head()

date
2017-02-28    0.000000
2017-03-31    0.078550
2017-04-30    0.012830
2017-05-31    0.061119
2017-06-30    0.053078
Freq: ME, Name: small_return, dtype: float64

### 코스피 월간 수익률 게산

In [20]:
kospi = data_loader.load_index_data(ticker_list=['1001'], freq='m', delay=1)
kospi_returns = kospi['close'].pct_change().loc[simulation_fromdate:]
kospi_returns.iloc[0] = 0.0
kospi_returns.name = 'kospi_return'
kospi_returns.index.name = 'date'
kospi_returns.head()

date
2017-02-28    0.000000
2017-03-31    0.032792
2017-04-30    0.020928
2017-05-31    0.064359
2017-06-30    0.018919
Freq: ME, Name: kospi_return, dtype: float64

### 4가지 평가 지표

In [28]:
print(f'per cagr: {cagr(returns=per_returns, freq='m'): .4f}')
print(f'pbr cagr: {cagr(returns=pbr_returns, freq='m'): .4f}')
print(f'kospi cagr: {cagr(returns=kospi_returns, freq='m'): .4f}')
print(f'dividend cagr: {cagr(returns=dividend_returns, freq='m'): .4f}')
print(f'small cagr: {cagr(returns=small_returns, freq='m'): .4f}')

per cagr:  0.1294
pbr cagr:  0.0358
kospi cagr:  0.0745
dividend cagr:  0.1150
small cagr:  0.3583


In [29]:
print(f'per mdd: {mdd(returns=per_returns): .4f}')
print(f'pbr mdd: {mdd(returns=pbr_returns): .4f}')
print(f'kospi mdd: {mdd(returns=kospi_returns): .4f}')
print(f'small mdd: {mdd(returns=small_returns): .4f}')

per mdd: -0.2209
pbr mdd: -0.4314
kospi mdd: -0.3163
small mdd: -0.2693


In [30]:
print(f'per sharpe_ratio: {sharpe_ratio(returns=per_returns, freq='m'):.4f}')
print(f'pbr sharpe_ratio: {sharpe_ratio(returns=pbr_returns, freq='m'):.4f}')
print(f'kospi sharpe_ratio: {sharpe_ratio(returns=kospi_returns, freq='m'):.4f}')
print(f'small sharpe_ratio: {sharpe_ratio(returns=small_returns, freq='m'):.4f}')

per sharpe_ratio: 0.6982
pbr sharpe_ratio: 0.2688
kospi sharpe_ratio: 0.5033
small sharpe_ratio: 1.2841


In [31]:
print(f'per sortino_ratio: {sortino_ratio(returns=per_returns, freq='m'): .4f}')
print(f'pbr sortino_ratio: {sortino_ratio(returns=pbr_returns, freq='m'): .4f}')
print(f'kospi sortino_ratio: {sortino_ratio(returns=kospi_returns, freq='m'): .4f}')
print(f'small sortino_ratio: {sortino_ratio(returns=small_returns, freq='m'): .4f}')

per sortino_ratio:  0.8293
pbr sortino_ratio:  0.2469
kospi sortino_ratio:  0.4913
small sortino_ratio:  1.5733


### 누적 수익률 곡선

In [32]:
benchmark_cum_returns = (kospi_returns + 1).cumprod() - 1
per_cum_returns = (per_returns + 1).cumprod() - 1
pbr_cum_returns = (pbr_returns + 1).cumprod() - 1
dividend_cum_returns = (dividend_returns + 1).cumprod() - 1
small_cum_returns = (small_returns + 1).cumprod() - 1

cum_returns = pd.concat([benchmark_cum_returns, per_cum_returns, pbr_cum_returns
                            , dividend_cum_returns, small_cum_returns], axis=1)
cum_returns.columns = ['코스피(KOSPI)', 'PER', 'PBR', '배당', '소형주']

fig = px.line(data_frame=cum_returns)

fig.update_xaxes(title_text='날짜')
fig.update_yaxes(title_text='누적 수익률')

fig.update_layout(legend_title_text='포트폴리오')

fig.show()